In [82]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math

torch.manual_seed(777)

basic attention structure 

In [83]:
def Attention(input):
    n = input.shape[0]
    d = input.shape[1]
    
    W_k = torch.randn(d, d)
    W_q = torch.randn(d, d)
    W_v = torch.randn(d, d)
    
    k = torch.matmul(input, W_k)
    q = torch.matmul(input, W_q)
    v = torch.matmul(input, W_v)
    # 여기까지 input으로 K, Q, V 만드는 과정 (embedding)
    
    # Attention
    
    #print(k, q, v)
    A = torch.zeros(n, n)
    root_d = math.sqrt(d)
    product = torch.matmul(q, k.transpose(0, 1))
    A = nn.functional.softmax(product / root_d, dim = 1)
    print(v, '\n', A)
    res = torch.matmul(A, v)
    return res

In [84]:
n = torch.randn(2, 3)
print(n)
Attention(n)

tensor([[-0.4015, -0.5934,  1.6885],
        [ 0.5554, -0.9433, -0.6758]])
tensor([[-2.4831,  0.2334,  1.9331],
        [ 2.1000,  4.8312, -1.2138]]) 
 tensor([[0.9842, 0.0158],
        [0.4623, 0.5377]])


tensor([[-2.4109,  0.3059,  1.8835],
        [-0.0189,  2.7056,  0.2411]])

## Transformer

In [85]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention).__init__()
        
    def attention(self, q, k, v):
#         product = torch.matmul(q, k.transpose(1, 2))  # matmul (B, X, Y)
        product = torch.matmul(q, k.T)
        root_d = math.sqrt(self.d_k)
        A = nn.functional.softmax(product / root_d, dim = 1)
        res = torch.matmul(A, v)
        
        return res

In [114]:
class EncoderLayer(Attention):
    def __init__(self):
        super(EncoderLayer).__init__()
        Attention.__init__(self) # 확인하기
    
    def Multi_Head_Attention(self, K, V, Q):  
        res = []
        print('K.shape, V.shape, Q.shape: ', K.shape, V.shape, Q.shape)
        for i in range(self.num_head):
            head = self.attention(K[i], V[i], Q[i])
            print('head.shape: ', head.shape)
            res.append(head)
        res = torch.cat(res, dim = 1)
        # print(res.shape, self.W_o.shape)
        res = torch.matmul(res, self.W_o)
        
        return res
    
    def LayerNorm(self, x, sub_layer):
        # print(type(x), type(sub_layer))
        a = x + sub_layer
        return F.layer_norm(a, a.shape)
    
    def FFN(self, x):  # Position-wise feed forward network
        x = F.linear(x, self.W1, self.b1)
        x = F.relu(x)
        x = F.linear(x, self.W2, self.b2)
        return x
        
    def KVQ(self, x):
        K = torch.zeros( self.num_head, self.input_shape, self.d_k)
        V = torch.zeros( self.num_head, self.input_shape, self.d_k)
        Q = torch.zeros( self.num_head, self.input_shape, self.d_k)
        for i in range(self.num_head):
            K[i] = torch.matmul(x, self.W_k[i])
            V[i] = torch.matmul(x, self.W_v[i])
            Q[i] = torch.matmul(x, self.W_q[i])
            
        return K, V, Q
    
    def singleLayer(self, x):
        self.K, self.V, self.Q = self.KVQ(x)
        print(self.K, self.V, self.Q)
        res = torch.Tensor()
        res = self.Multi_Head_Attention(self.K, self.V, self.Q)  # Q = [q1, q2, ....] where dim(qi) = (input_shape, d_k)
        print(res)
        res = self.LayerNorm(x, res)
        
        # Fully connected feed forward network
        print('res.shape: ', res.shape)
        fc = self.FFN(res)
        
        res = self.LayerNorm(res, fc)
        
        return res
        

In [115]:
class Encoder(EncoderLayer):
    def __init__(self, N, input_shape, d_k, at_dim, d_ff, d_model, dropout_rate, num_head, W_k, W_q, W_v, W_o, W1, W2, b1, b2):
        super(Encoder).__init__()
        
        EncoderLayer.__init__(self)
        # store hyperparameters
        self.N = N
        self.input_shape = input_shape
        self.d_k = d_k
        self.at_dim = at_dim
        self.d_ff = d_ff
        self.d_model = d_model
        self.dropout_rate = dropout_rate
        self.num_head = num_head
        self.W_k = W_k
        self.W_q = W_q
        self.W_v = W_v
        self.W_o = W_o
        self.W1 = W1
        self.W2 = W2
        self.b1 = b1
        self.b2 = b2
    def forward(self, x):
        res = x
        # N encoder layers
        for i in range(self.N):
            res = self.singleLayer(res)
        
        return res    

In [116]:
inp = torch.randn(1, 2, 2)

In [117]:
a = torch.Tensor([[[1, 2], [2, 3]], [[4, 5], [8, 3]], [[4, 1], [7, 8]]])
b = torch.randn(6, 2)
weight = torch.randn(2, 2)
model = Encoder(2, 2, 2, 2, 2, 2, 0.2, 3, a, a, a, b, weight, weight, torch.tensor([[1]]), torch.tensor([[2]]))
inp = torch.Tensor([[10, 10], [20, 20]])
model.forward(inp.unsqueeze(0))


tensor([[[ 30.,  50.],
         [ 60., 100.]],

        [[120.,  80.],
         [240., 160.]],

        [[110.,  90.],
         [220., 180.]]]) tensor([[[ 30.,  50.],
         [ 60., 100.]],

        [[120.,  80.],
         [240., 160.]],

        [[110.,  90.],
         [220., 180.]]]) tensor([[[ 30.,  50.],
         [ 60., 100.]],

        [[120.,  80.],
         [240., 160.]],

        [[110.,  90.],
         [220., 180.]]])
K.shape, V.shape, Q.shape:  torch.Size([3, 2, 2]) torch.Size([3, 2, 2]) torch.Size([3, 2, 2])
head.shape:  torch.Size([2, 2])
head.shape:  torch.Size([2, 2])
head.shape:  torch.Size([2, 2])
tensor([[ 148.3409, -210.9272],
        [ 148.3409, -210.9272]])
res.shape:  torch.Size([1, 2, 2])
tensor([[[-1.0292, -1.0502],
         [-0.9706, -0.9497]],

        [[-4.1170,  1.9121],
         [-3.8826,  2.0877]],

        [[-3.1086, -7.0792],
         [-2.8910, -6.9200]]]) tensor([[[-1.0292, -1.0502],
         [-0.9706, -0.9497]],

        [[-4.1170,  1.9121],
         [

tensor([[[ 0.9909, -1.0016],
         [ 1.0090, -0.9983]]])